In [1]:
%load_ext autoreload
%autoreload 2

In [68]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm

client = OpenAI(
    api_key=os.getenv("OPENAI_KEY"),
)

MODEL_NAME = "gpt-4"

In [75]:
country_title = pd.read_csv("../data/raw/country_title.csv")
country_title_unique = country_title.drop_duplicates(subset=["country"])

print(f"filtered from {country_title.shape[0]} to {country_title_unique.shape[0]}")

progress = tqdm(country_title_unique.iterrows(), desc = "Generating", total = country_title_unique.shape[0]) 

print("Generating ...")

for index, row in progress:
    country = row["country"]
    role = row["title"]
    progress.set_description(f"{country} - {role}")

    prompt = f'''Give me the names of {role}s of {country} in csv till the year 2022. The format is below, please add the names of the column as well.
    name, start year, end year
    '''

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
            # {"role": "user", "content": "Give me 10 words in english with their german translations."},
        ],
        temperature=0,
        max_tokens=7500,
    )

    with open(f"../data/raw/country_leaders/{country}.csv", "w") as f:
        f.write(response.choices[0].message.content)

    time.sleep(2)

filtered from 234 to 148


Generating:   0%|          | 0/148 [00:00<?, ?it/s]

Generating ...


In [32]:
country = "Bangladesh"
role = "Prime Minister"

prompt = f'''Give me the names of {role}s of {country} in csv till the year 2022. The format is below, please add the names of the column as well.
name, start year, end year
'''

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
        # {"role": "user", "content": "Give me 10 words in english with their german translations."},
    ],
    temperature=0,
    max_tokens=3500,
)

In [33]:
print(response.choices[0].message.content)

name, start year, end year
Tajuddin Ahmad, 1971, 1972
Muhammad Mansur Ali, 1972, 1975
Mujibur Rahman, 1975, 1975
Shah Azizur Rahman, 1977, 1981
Ataur Rahman Khan, 1981, 1982
Abdus Sattar, 1982, 1982
Shah Azizur Rahman, 1982, 1982
Kazi Zafar Ahmed, 1988, 1991
Begum Khaleda Zia, 1991, 1996
Hasina Wazed, 1996, 2001
Begum Khaleda Zia, 2001, 2006
Fakhruddin Ahmed, 2007, 2009
Hasina Wazed, 2009, 2022


In [72]:
with open("../data/raw/country_leaders/bd.csv", "w") as f:
    f.write(response.choices[0].message.content)

In [ ]:
pd.read_csv